In [ ]:
# 初回だけ実行
# %pip install pandas
# %pip install wavedrom
# %pip install svgwrite
# %pip install rp2040-pio-emulator@git+https://github.com/NathanY3G/rp2040-pio-emulator
# %pip install adafruit-circuitpython-pioasm

In [1]:
from dataclasses import dataclass
import json
from typing import Any, Callable, Dict, List, Optional
import pandas as pd
from collections import deque
import itertools
import adafruit_pioasm
import pioemu
import wavedrom
import svgwrite



@dataclass
class EmulateResult:
    """PIOのエミュレーション結果を格納するクラス"""

    program_str: str
    test_cycles: int
    tx_fifo_entries: List[int]
    run_states_df: pd.DataFrame
    rx_fifo: List[int]
    wavedrom_src: str
    wave_svg: svgwrite.drawing.Drawing


def to_hex_u32(x: int) -> str:
    """32bit符号なし整数を16進数文字列に変換する"""
    return f"0x{(x& 0xffffffff):08X}"


def emulate_pio(
    program_str: str, test_cycles: int, tx_fifo_entries: List[int] = [], dequeue_period_cyc: int = 0
) -> EmulateResult:
    """PIOのエミュレーションを行う"""

    # emulation
    opcodes = adafruit_pioasm.assemble(program_str)
    test_cycles = 100
    generator = pioemu.emulate(
        opcodes=opcodes,
        stop_when=lambda _, state: state.clock > test_cycles,
        initial_state=pioemu.State(
            clock=0,
            program_counter=0,
            transmit_fifo=deque(tx_fifo_entries),
            receive_fifo=deque([]),
            x_register=0,
            y_register=0,
        ),
        shift_isr_right=True,
        shift_osr_right=True,
    )

    # run
    run_states = []
    dma_dequeue_ready_cnt = 0 # dequeue_period_cycより大きければDequeue可能
    for before, after in itertools.islice(generator, test_cycles):
        run_states.append(after.__dict__)
        # dequeue
        dma_dequeue_ready_cnt += 1
        if (dma_dequeue_ready_cnt > dequeue_period_cyc) and (len(before.receive_fifo) > 0):
            after.receive_fifo.popleft()
            dma_dequeue_ready_cnt = 0


    run_states_df = pd.DataFrame.from_records(run_states)
    rx_fifo = list(run_states_df[-1:]["receive_fifo"].values[0])

    # 欲しいデータ抽出

    def get_insert_idx() -> int:
        """DataFrameに新しい列を挿入するためのインデックスを返す"""

        dst = get_insert_idx.insert_idx  # increment前を返す
        get_insert_idx.insert_idx += 1
        return dst

    get_insert_idx.insert_idx = 0

    run_states_df.insert(
        get_insert_idx(),
        "cyc",
        run_states_df["clock"],
    )
    run_states_df.insert(
        get_insert_idx(),
        "pc",
        run_states_df["program_counter"],
    )
    run_states_df.insert(
        get_insert_idx(),
        "inst",
        run_states_df["pc"].map(lambda pc: to_hex_u32(opcodes[pc])),
    )
    run_states_df.insert(
        get_insert_idx(), "x", run_states_df["x_register"].map(to_hex_u32)
    )
    run_states_df.insert(
        get_insert_idx(), "y", run_states_df["y_register"].map(to_hex_u32)
    )
    run_states_df.insert(
        get_insert_idx(),
        "isr",
        run_states_df["input_shift_register"].map(lambda sr: to_hex_u32(sr.contents)),
    )
    run_states_df.insert(
        get_insert_idx(),
        "osr",
        run_states_df["output_shift_register"].map(lambda sr: to_hex_u32(sr.contents)),
    )
    run_states_df.insert(
        get_insert_idx(), "pindirs", run_states_df["pin_directions"].map(to_hex_u32)
    )
    run_states_df.insert(
        get_insert_idx(), "pins", run_states_df["pin_values"].map(to_hex_u32)
    )
    run_states_df.insert(
        get_insert_idx(),
        "io",
        run_states_df["pin_values"].map(
            lambda data: to_hex_u32((data & 0x000000FF) >> 0)
        ),
    )
    run_states_df.insert(
        get_insert_idx(),
        "ceb0",
        run_states_df["pin_values"].map(lambda data: (data & 0x00000100) >> 8),
    )
    run_states_df.insert(
        get_insert_idx(),
        "ceb1",
        run_states_df["pin_values"].map(lambda data: (data & 0x00000200) >> 9),
    )
    run_states_df.insert(
        get_insert_idx(),
        "cle",
        run_states_df["pin_values"].map(lambda data: (data & 0x00000400) >> 10),
    )
    run_states_df.insert(
        get_insert_idx(),
        "ale",
        run_states_df["pin_values"].map(lambda data: (data & 0x00000800) >> 11),
    )
    run_states_df.insert(
        get_insert_idx(),
        "wpb",
        run_states_df["pin_values"].map(lambda data: (data & 0x00001000) >> 12),
    )
    run_states_df.insert(
        get_insert_idx(),
        "web",
        run_states_df["pin_values"].map(lambda data: (data & 0x00002000) >> 13),
    )
    run_states_df.insert(
        get_insert_idx(),
        "reb",
        run_states_df["pin_values"].map(lambda data: (data & 0x00004000) >> 14),
    )
    run_states_df.insert(
        get_insert_idx(),
        "rbb",
        run_states_df["pin_values"].map(lambda data: (data & 0x00008000) >> 15),
    )
    run_states_df.insert(
        get_insert_idx(),
        "txfifo_head",
        run_states_df["transmit_fifo"].map(
            lambda data: to_hex_u32(data[0]) if len(data) > 0 else None
        ),
    )
    run_states_df.insert(
        get_insert_idx(),
        "txfifo_remain",
        run_states_df["transmit_fifo"].map(lambda data: len(data)),
    )
    run_states_df.insert(
        get_insert_idx(),
        "rxfifo_tail",
        run_states_df["receive_fifo"].map(
            lambda data: to_hex_u32(data[-1]) if len(data) > 0 else None
        ),
    )
    run_states_df.insert(
        get_insert_idx(),
        "rxfifo_remain",
        run_states_df["receive_fifo"].map(lambda data: len(data)),
    )

    # シーケンス解析
    run_states_df["cs_assert"] = (run_states_df["ceb0"] == 0) | (
        run_states_df["ceb1"] == 0
    )
    run_states_df["web_edge"] = (run_states_df["web"] == 1) & (
        run_states_df.shift(1)["web"] == 0
    )  # rising edge
    run_states_df["reb_edge"] = (run_states_df["reb"] == 0) & (
        run_states_df.shift(1)["reb"] == 1
    )  # falling edge
    run_states_df.insert(
        get_insert_idx(),
        "cmd_in",
        run_states_df["web_edge"]
        & run_states_df["cs_assert"]
        & (run_states_df["reb"] == 1)
        & (run_states_df["cle"] == 1)
        & (run_states_df["ale"] == 0),
    )
    run_states_df.insert(
        get_insert_idx(),
        "addr_in",
        run_states_df["web_edge"]
        & run_states_df["cs_assert"]
        & (run_states_df["reb"] == 1)
        & (run_states_df["cle"] == 0)
        & (run_states_df["ale"] == 1),
    )
    run_states_df.insert(
        get_insert_idx(),
        "data_in",
        run_states_df["web_edge"]
        & run_states_df["cs_assert"]
        & (run_states_df["reb"] == 1)
        & (run_states_df["cle"] == 0)
        & (run_states_df["ale"] == 0),
    )
    run_states_df.insert(
        get_insert_idx(),
        "data_out",
        run_states_df["reb_edge"]
        & run_states_df["cs_assert"]
        & (run_states_df["web"] == 1)
        & (run_states_df["cle"] == 0)
        & (run_states_df["ale"] == 0),
    )

    def to_wavedrom_signal(
        df: pd.DataFrame,
        col: str,
        replace_f: Optional[Callable[[Any, Any], Any]] = None,
    ) -> Dict[str, Any]:
        """DataFrameの列をWavedromの信号に変換する"""
        src = df[col].map(lambda x: "1" if x else "0").to_list()
        dst_wave = []

        prev_data = None
        for entry in src:
            # 差し替え関数あるなら任せる
            if replace_f is not None:
                dst_wave.append(replace_f(prev_data, entry))
            elif prev_data == entry:
                dst_wave.append(".")
            else:
                dst_wave.append(entry)
            # 一つ前のデータを保存
            prev_data = entry
        return {"name": col, "wave": "".join(dst_wave)}

    def to_wavedrom_data(df: pd.DataFrame, col: str) -> Dict[str, Any]:
        """DataFrameの列をWavedromの信号に変換する"""
        src = df[col].to_list()
        dst_wave = []
        dst_data = []

        prev_data = None
        for entry in src:
            # Noneのときは最初から一致してしまう
            if entry is None:
                dst_wave.append("x")
            elif prev_data == entry:
                dst_wave.append(".")
            else:
                dst_wave.append("=")
                dst_data.append(
                    hex(
                        int(entry, 16),
                    ).replace("0x", "")
                    if isinstance(entry, str)
                    else entry
                )  # 長いので縮める, str|intの場合分け
            # 一つ前のデータを保存
            prev_data = entry
        return {"name": col, "wave": "".join(dst_wave), "data": dst_data}

    wavedrom_src = json.dumps(
        {
            "signal": [
                [
                    "pio",
                    [
                        "ctrl",
                        to_wavedrom_data(run_states_df, "cyc"),
                        to_wavedrom_data(run_states_df, "pc"),
                        to_wavedrom_data(run_states_df, "inst"),
                        [
                            "fifo",
                            [
                                "tx",
                                to_wavedrom_data(run_states_df, "txfifo_head"),
                                to_wavedrom_data(run_states_df, "txfifo_remain"),
                            ],
                            [
                                "rx",
                                to_wavedrom_data(run_states_df, "rxfifo_tail"),
                                to_wavedrom_data(run_states_df, "rxfifo_remain"),
                            ],
                        ],
                    ],
                    {},
                    [
                        "regs",
                        [
                            "scratch",
                            to_wavedrom_data(run_states_df, "x"),
                            to_wavedrom_data(run_states_df, "y"),
                        ],
                        [
                            "fifo",
                            to_wavedrom_data(run_states_df, "isr"),
                            to_wavedrom_data(run_states_df, "osr"),
                        ],
                        [
                            "pinout",
                            to_wavedrom_data(run_states_df, "pindirs"),
                            to_wavedrom_data(run_states_df, "pins"),
                        ],
                    ],
                ],
                {},
                [
                    "nand",
                    [
                        "out",
                        [
                            "cs",
                            to_wavedrom_signal(run_states_df, "ceb0"),
                            to_wavedrom_signal(run_states_df, "ceb1"),
                        ],
                        [
                            "latch",
                            to_wavedrom_signal(run_states_df, "cle"),
                            to_wavedrom_signal(run_states_df, "ale"),
                        ],
                        [
                            "edge",
                            to_wavedrom_signal(run_states_df, "web"),
                            to_wavedrom_signal(run_states_df, "reb"),
                        ],
                        to_wavedrom_signal(run_states_df, "wpb"),
                    ],
                    {},
                    [
                        "inout",
                        to_wavedrom_data(run_states_df, "io"),
                    ],
                    {},
                    [
                        "in",
                        to_wavedrom_signal(run_states_df, "rbb"),
                    ],
                    {},
                    [
                        "analysis",
                        to_wavedrom_signal(run_states_df, "cs_assert"),
                        to_wavedrom_signal(run_states_df, "web_edge"),
                        to_wavedrom_signal(run_states_df, "reb_edge"),
                        to_wavedrom_signal(run_states_df, "cmd_in"),
                        to_wavedrom_signal(run_states_df, "addr_in"),
                        to_wavedrom_signal(run_states_df, "data_in"),
                        to_wavedrom_signal(run_states_df, "data_out"),
                    ],
                ],
            ]
        }
    )
    wave_svg = wavedrom.render(wavedrom_src)

    return EmulateResult(
        program_str=program_str,
        test_cycles=test_cycles,
        tx_fifo_entries=tx_fifo_entries,
        run_states_df=run_states_df,
        rx_fifo=rx_fifo,
        wavedrom_src=wavedrom_src,
        wave_svg=wave_svg,
    )


In [2]:
from typing import List, Optional
import pandas as pd
from enum import Enum, auto
from IPython.display import display
from pathlib import Path

#################################################
# NAND ICで決められた定義


class NandPinAssign(Enum):
    """
    NAND IC Pinout
    | 15  | 14  | 13  | 12  | 11  | 10  | 9    | 8    | 7   | 6   | 5   | 4   | 3   | 2   | 1   | 0   |
    | --- | --- | --- | --- | --- | --- | ---- | ---- | --- | --- | --- | --- | --- | --- | --- | --- |
    | rbb | reb | web | wpb | ale | cle | ceb1 | ceb0 | io7 | io6 | io5 | io4 | io3 | io2 | io1 | io0 |
    | in  | out | out | out | out | out | out  | out  | io  | io  | io  | io  | io  | io  | io  | io  |
    """

    Io0 = 0
    Io1 = 1
    Io2 = 2
    Io3 = 3
    Io4 = 4
    Io5 = 5
    Io6 = 6
    Io7 = 7
    Ceb0 = 8
    Ceb1 = 9
    Cle = 10
    Ale = 11
    Wpb = 12
    Web = 13
    Reb = 14
    Rbb = 15
    Led = 25


class NandFlashCommand(Enum):
    """NAND Flash Command"""

    Reset = 0xFF
    ReadId = 0x90


def bit_on(bit_pos: int) -> int:
    """指定したbitだけ1の値"""
    return 0x01 << bit_pos


def combine_halfword(low: int, high: int) -> int:
    """2byteの値を結合する"""
    return (high << 16) | low


def gen_ceb_bits(select_cs: Optional[int] = None) -> int:
    """cs指定からCEB0/CEB1のピン状態を返す"""
    if select_cs is None:
        return bit_on(NandPinAssign.Ceb0.value) | bit_on(NandPinAssign.Ceb1.value)
    elif select_cs == 0:
        return bit_on(NandPinAssign.Ceb1.value)
    elif select_cs == 1:
        return bit_on(NandPinAssign.Ceb0.value)
    else:
        raise ValueError("select_cs must be 0 or 1 or None")


# RBB以外全部Output
write_pin_dir = (
    bit_on(NandPinAssign.Reb.value)
    | bit_on(NandPinAssign.Web.value)
    | bit_on(NandPinAssign.Ale.value)
    | bit_on(NandPinAssign.Cle.value)
    | bit_on(NandPinAssign.Ceb1.value)
    | bit_on(NandPinAssign.Ceb0.value)
    | bit_on(NandPinAssign.Io7.value)
    | bit_on(NandPinAssign.Io6.value)
    | bit_on(NandPinAssign.Io5.value)
    | bit_on(NandPinAssign.Io4.value)
    | bit_on(NandPinAssign.Io3.value)
    | bit_on(NandPinAssign.Io2.value)
    | bit_on(NandPinAssign.Io1.value)
    | bit_on(NandPinAssign.Io0.value)
)
disable_write_protect = bit_on(NandPinAssign.Wpb.value)

# RBB,IO以外Output
read_pin_dir = (
    bit_on(NandPinAssign.Reb.value)
    | bit_on(NandPinAssign.Web.value)
    | bit_on(NandPinAssign.Wpb.value)
    | bit_on(NandPinAssign.Ale.value)
    | bit_on(NandPinAssign.Cle.value)
    | bit_on(NandPinAssign.Ceb1.value)
    | bit_on(NandPinAssign.Ceb0.value)
)

# Assert=/WP, Negate=/CS,/RE,/WE,/CLE,ALE,
init_pin_state = (
    bit_on(NandPinAssign.Reb.value)
    | bit_on(NandPinAssign.Web.value)
    | bit_on(NandPinAssign.Ceb1.value)
    | bit_on(NandPinAssign.Ceb0.value)
)

# cs=1対応する場合は切り替え
cs_assert = gen_ceb_bits(select_cs=0)

# /RE, /WE無効, CLE, ALE 無効
init_pin_with_cs = (
    bit_on(NandPinAssign.Reb.value) | bit_on(NandPinAssign.Web.value) | cs_assert
)
# edge生成用にL/Hを組み合わせて使う
cmd_latch_0 = bit_on(NandPinAssign.Cle.value) | bit_on(NandPinAssign.Reb.value)  # CLE
cmd_latch_1 = (
    bit_on(NandPinAssign.Cle.value)
    | bit_on(NandPinAssign.Reb.value)
    | bit_on(NandPinAssign.Web.value)
)  # CLE + posedge /WE
addr_latch_0 = bit_on(NandPinAssign.Ale.value) | bit_on(NandPinAssign.Reb.value)  # ALE
addr_latch_1 = (
    bit_on(NandPinAssign.Ale.value)
    | bit_on(NandPinAssign.Reb.value)
    | bit_on(NandPinAssign.Web.value)
)  # ALE + posedge /WE
data_in_latch_0 = bit_on(NandPinAssign.Reb.value)
data_in_latch_1 = bit_on(NandPinAssign.Reb.value) | bit_on(
    NandPinAssign.Web.value
)  # posedge /WE
data_out_latch_0 = bit_on(NandPinAssign.Web.value)  # negedge /RE (@NAND IC Dataout)
data_out_latch_1 = bit_on(NandPinAssign.Web.value) | bit_on(
    NandPinAssign.Reb.value
)  # posedge /RE (capture data)

#################################################
# broccoli nandio 独自の定義


class BroccoliNandIoCommand(Enum):
    """Broccoli NAND IO Command"""

    Bitbang = auto()
    CmdLatch = auto()
    AddrLatch = auto()
    DataOut = auto()
    DataIn = auto()
    SetIrq = auto()
    WaitRbb = auto()


def gen_bnio_cmd(cmd: BroccoliNandIoCommand, pindirs: int, transfer_count: int) -> int:
    """コマンドの先頭wordを返す. RShiftで取り出す想定. transfer_countに実際にセットされる値は、pio都合で-1される。"""
    return ((cmd.value & 0xF) << 28) | (
        (((transfer_count - 1) & 0x0FFF) << 16) | (pindirs & 0xFFFF)
    )


# TODO: DSL的なものがあってもいいかも。ある程度動くようになってからリファクタする
tx_fifo_entries = [
    #############################################
    ### CMD: RESET ###
    gen_bnio_cmd(
        cmd=BroccoliNandIoCommand.Bitbang, pindirs=write_pin_dir, transfer_count=2
    ),
    # init pin state -> Reset Cmd -> init pin state
    combine_halfword(
        low=init_pin_state,
        high=NandFlashCommand.Reset.value | cmd_latch_0 | cs_assert,
    ),
    combine_halfword(
        low=NandFlashCommand.Reset.value | cmd_latch_1 | cs_assert,
        high=init_pin_state,
    ),
    #############################################
    ### CMD: READ ID ###
    gen_bnio_cmd(
        cmd=BroccoliNandIoCommand.Bitbang, pindirs=write_pin_dir, transfer_count=3
    ),
    # init pin state -> Read ID Cmd -> Address 00 -> init pin state
    combine_halfword(
        low=init_pin_state,
        high=NandFlashCommand.ReadId.value | cmd_latch_0 | cs_assert,
    ),
    combine_halfword(
        low=NandFlashCommand.ReadId.value | cmd_latch_1 | cs_assert,
        high=0x00 | addr_latch_0 | cs_assert,
    ),
    combine_halfword(
        low=0x00 | addr_latch_1 | cs_assert,
        high=init_pin_with_cs,  # readに続けるのでCSは維持
    ),
    #############################################
    ### Read: 5byte ###
    gen_bnio_cmd(
        cmd=BroccoliNandIoCommand.Read, pindirs=read_pin_dir, transfer_count=5
    ),
    combine_halfword(
        low=bit_on(NandPinAssign.Web.value) | cs_assert,  # data_read_en
        high=bit_on(NandPinAssign.Reb.value)
        | bit_on(NandPinAssign.Web.value)
        | cs_assert,  # data_read_dis
    ),
    #############################################
    ### init pin ###
    gen_bnio_cmd(
        cmd=BroccoliNandIoCommand.Bitbang, pindirs=write_pin_dir, transfer_count=1
    ),
    # init pin state
    combine_halfword(
        low=init_pin_state,
        high=init_pin_state,
    ),
]

# PIO Assembler
program = Path("broccoli-nandio.pio").read_text(encoding="utf-8")
result = emulate_pio(
    program, test_cycles=100, tx_fifo_entries=tx_fifo_entries, dequeue_period_cyc=16
)

print(
    f"tx_fifo_entries: {[str(i) + ':' + hex(x) for i, x in enumerate(result.tx_fifo_entries)]}"
)
print(f"rx_fifo: {result.rx_fifo}")
print(f"wavedrom_src: {result.wavedrom_src}")

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

# output logs
Path("dst").mkdir(exist_ok=True)
Path("dst/tx_fifo.json").write_text(
    json.dumps(result.tx_fifo_entries), encoding="utf-8"
)
Path("dst/rx_fifo.json").write_text(json.dumps(result.rx_fifo), encoding="utf-8")
Path("dst/wave.json").write_text(result.wavedrom_src)
result.run_states_df.to_csv("dst/run_states.csv")
result.wave_svg.saveas("dst/wave.svg")

display(result.wave_svg)
display(result.run_states_df.query("cs_assert"))

AttributeError: type object 'BroccoliNandIoCommand' has no attribute 'Read'